In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Intro

Welcome to the BlendingToolKit (BTK) quickstart! This tutorial will guide you through the basic functionalities of BTK. If you are new to BTK, we recommend you to go through this notebook before moving on to the other tutorials.

We will show you how to generate a catalog of galaxy blends, draw images of these blends, use algorithms implemented within BTK (e.g. SExtractor) to deblend these images, and finally use metrics within BTK to assess the performance of the deblender.

# Setup

First, we will need to import the relevant python packages and modules:

In [ ]:
%matplotlib inline
import numpy as np
import os

In [ ]:
import btk
import btk.survey
import btk.draw_blends
import btk.catalog
import btk.sampling_functions
import astropy.table

# Drawing Blends

Before generating images of blends, we require some setup. 

**First**, we need to specify a catalog of galaxies to draw. We will use a subset of the CATSIM catalog, which contains `100` galaxies with realistic parameters.

BTK uses a wrapper class (e.g. `btk.catalog.CatsimCatalog`) to store information about the galaxy catalog you pass in. You can easily import the CATSIM catalog from a FITS file using the `from_file` method as demonstrated here with our example catalog:

In [ ]:
catalog_name = "../data/sample_input_catalog.fits"
catalog = btk.catalog.CatsimCatalog.from_file(catalog_name)
catalog.table[:5] # display 5 first entries of table containing the actual catalog information.

galtileid,ra,dec,redshift,fluxnorm_bulge,fluxnorm_disk,fluxnorm_agn,a_b,a_d,b_b,b_d,pa_bulge,pa_disk,u_ab,g_ab,r_ab,i_ab,z_ab,y_ab,ref_mag,btk_size
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2200871446,1505.9509277292,-0.5342399817876,0.496377289295,0.0,1.4144730572e-17,0.0,0.0,0.278649687767,0.0,0.221303001046,0.0,307.344329834,25.9418621063,25.129743576,23.9588813782,23.3607368469,23.0723800659,22.9095973969,23.3607368469,1.29850754499422
2205921112,1512.1024131744,-3.609359823168,1.89508104324,0.0,1.91501907101e-18,0.0,0.0,0.358063697815,0.0,0.313674807549,0.0,137.791702271,25.848903656,25.867565155,25.9179477692,25.9851398468,25.8779563904,25.7642536163,25.9851398468,1.6685768318179002
2205010878,1524.3564605652,-11.266919877384,1.93795013428,0.0,2.15612608955e-18,0.0,0.0,0.444279909134,0.0,0.424689114094,0.0,233.972427368,25.5657653809,25.5659580231,25.6165962219,25.6957893372,25.6281528473,25.5399188995,25.6957893372,2.07034437656444
2208734677,1515.910291668,-0.26135999942183996,1.16251754761,0.0,1.74001598433e-18,0.0,0.0,0.312852591276,0.0,0.180962398648,0.0,217.517120361,27.3159255981,27.4167633057,27.051820755,26.6737632751,25.9380722046,25.6458129883,26.6737632751,1.45789307534616
2212241881,1523.4518051136,-7.851959858088,1.35479903221,0.0,1.11031595423e-18,0.0,0.0,0.414316505194,0.0,0.205554202199,0.0,226.523849487,27.3325939178,27.4470024109,27.3282527924,27.0680370331,26.7419490814,26.1817016602,27.0680370331,1.93071491420404


**Second**, we require specifying a *sampling function*. The sampling function is a custom class within BTK that is used to determine which galaxies are drawn for each blend and what their locations within the blend should be. 

Specifically, we implement this functionality with the `SamplingFunction` class, which is a `callable` like a function. It takes as argument the astropy table contained within the `Catalog` object above, selects galaxies to be drawn in a given blend, and returns their (possibly modified) entries. Usually the `ra` and `dec` columns of these entries are modified to be relative to the center of the postage stamp containing the blend. 

For this tutorial, we will use the *default sampling function*, this function is implemented in the `btk.sampling_functions` module and is called `DefaultSampling`. This function draws a random number of galaxies from the catalog, uniformly distributed between 1 and `max_number`, and places them randomly within the postage stamp with some maximum shift `max_shift` w.r.t the center of the postage stamp.

The `DefaultSampling` function can be instantiated as follows:

In [ ]:
stamp_size = 24.0  # Size of the stamp, in arcseconds
max_number = 5     # Maximum number of galaxies in a blend
max_shift = 3.0    # Maximum shift of the galaxies, in arcseconds
seed = 1 # random seed for reproducibility purposes
sampling_function = btk.sampling_functions.DefaultSampling(max_number=max_number, stamp_size=stamp_size, max_shift=max_shift, seed = seed)

Here is an example output of the `DefaultSampling` function:

In [ ]:
blend_cat = sampling_function(catalog.table)
blend_cat # ra and dec are now relative to the center of the blend (not the original ones)

galtileid,ra,dec,redshift,fluxnorm_bulge,fluxnorm_disk,fluxnorm_agn,a_b,a_d,b_b,b_d,pa_bulge,pa_disk,u_ab,g_ab,r_ab,i_ab,z_ab,y_ab,ref_mag,btk_size
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2201661997,0.2288598793156691,-0.27901266311609074,2.06801533699,1.09825997775e-19,1.67955797375e-17,0.0,0.208737000823,0.378724396229,0.18357090652,0.122722901404,219.0519104,219.0519104,24.6160621643,23.9625110626,24.1768531799,24.0236854553,23.7659263611,23.7034015656,24.0236854553,1.7592851415402924
2201104165,-1.021609701005447,-2.1957498165170115,0.812159895897,6.57883086866e-18,2.26453001134e-17,0.0,0.22702370584,0.191140606999,0.20540009439,0.0433686003089,24.3779182434,24.3779182434,25.9493045807,25.4186573029,24.5099773407,23.527223587,23.0481529236,22.813867569,23.527223587,0.7996899312496969
2200871446,1.7305722205704264,-0.5813220813172246,0.496377289295,0.0,1.4144730572e-17,0.0,0.0,0.278649687767,0.0,0.221303001046,0.0,307.344329834,25.9418621063,25.129743576,23.9588813782,23.3607368469,23.0723800659,22.9095973969,23.3607368469,1.29850754499422
2203664633,-1.18083102425013,-1.7792685559431023,1.43826675415,2.87464908402e-18,6.82620512451e-19,0.0,0.326700508595,0.0907427966595,0.279465585947,0.0626740008593,271.91583252,271.91583252,24.9983882904,25.0850524902,25.1815986633,25.1274318695,25.02277565,24.5629825592,25.1274318695,0.4670836444149558


In [ ]:
blend_cat2 = blend_cat.copy()
blend_cat2['ra'] = blend_cat2['ra'] + np.random.uniform(-0.5, 0.5, len(blend_cat2))
blend_cat2['dec'] = blend_cat2['dec'] + np.random.uniform(-0.5, 0.5, len(blend_cat2))


In [ ]:
blend_cat2

galtileid,ra,dec,redshift,fluxnorm_bulge,fluxnorm_disk,fluxnorm_agn,a_b,a_d,b_b,b_d,pa_bulge,pa_disk,u_ab,g_ab,r_ab,i_ab,z_ab,y_ab,ref_mag,btk_size
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2201661997,0.45204015035279743,-0.4197718435001935,2.06801533699,1.09825997775e-19,1.67955797375e-17,0.0,0.208737000823,0.378724396229,0.18357090652,0.122722901404,219.0519104,219.0519104,24.6160621643,23.9625110626,24.1768531799,24.0236854553,23.7659263611,23.7034015656,24.0236854553,1.7592851415402924
2201104165,-0.5931599208822096,-2.260963093635469,0.812159895897,6.57883086866e-18,2.26453001134e-17,0.0,0.22702370584,0.191140606999,0.20540009439,0.0433686003089,24.3779182434,24.3779182434,25.9493045807,25.4186573029,24.5099773407,23.527223587,23.0481529236,22.813867569,23.527223587,0.7996899312496969
2200871446,1.5173985052421237,-0.8782461046906186,0.496377289295,0.0,1.4144730572e-17,0.0,0.0,0.278649687767,0.0,0.221303001046,0.0,307.344329834,25.9418621063,25.129743576,23.9588813782,23.3607368469,23.0723800659,22.9095973969,23.3607368469,1.29850754499422
2203664633,-1.308164643247067,-1.7473728050433674,1.43826675415,2.87464908402e-18,6.82620512451e-19,0.0,0.326700508595,0.0907427966595,0.279465585947,0.0626740008593,271.91583252,271.91583252,24.9983882904,25.0850524902,25.1815986633,25.1274318695,25.02277565,24.5629825592,25.1274318695,0.4670836444149558


In [ ]:
from btk.utils import add_pixel_columns
add_pixel_columns()

## Matching

In [ ]:
from btk.match import pixel_l2_distance_matrix
from scipy.optimize import linear_sum_assignment
import numpy as np

In [ ]:
x1, y1 = np.array([1, 2, 3, 0.1]), np.array([1, 2, 3, 0.1])   
x2, y2 = np.array([0.2, 1.5, 0.5]), np.array([0.2, 1.5, 0.5])

s = np.array([1, 1, 20, 10, 0, 0, 0, 0])

In [ ]:
mat = pixel_l2_distance_matrix(x1, y1, x2, y2)
true_indx, pred_indx = linear_sum_assignment(mat)

In [ ]:
print(true_indx)
print(pred_indx)

[0 1 3]
[2 1 0]


In [ ]:
x1[true_indx], x2[pred_indx]

(array([1. , 2. , 0.1]), array([0.5, 1.5, 0.2]))

In [ ]:
matched_matrix = np.full(mat.shape, -1)
matched_matrix[true_indx, pred_indx] = pred_indx
matched_matrix

array([[-1, -1,  2],
       [-1,  1, -1],
       [-1, -1, -1],
       [ 0, -1, -1]])

In [ ]:
true_indx = [ii for ii, row in enumerate(matched_matrix) if np.any(row > -1)]
true_indx 

[0, 1, 3]

In [ ]:
s[true_indx]

array([ 1,  1, 10])

In [ ]:
pred_indx = [jj for jj,col in enumerate(matched_matrix.T) if np.any(col > -1)]
pred_indx

[0, 1, 2]